In [ ]:
pip install face_recognition

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
import face_recognition
import cv2
import numpy as np
import csv
import os
from datetime import datetime

video_capture = cv2.VideoCapture(0)

dhruv_image = face_recognition.load_image_file("Dhruv.JPG")
dhruv_encoding = face_recognition.face_encodings(dhruv_image)[0]

shan_image = face_recognition.load_image_file("Shan.JPG")
shan_encoding = face_recognition.face_encodings(shan_image)[0]

harsh_image = face_recognition.load_image_file("Harsh.JPG")
harsh_encoding = face_recognition.face_encodings(harsh_image)[0]

ujjwal_image = face_recognition.load_image_file("Ujjwal.JPG")
ujjwal_encoding = face_recognition.face_encodings(ujjwal_image)[0]

nishit_image = face_recognition.load_image_file("Nishit.JPG")
nishit_encoding = face_recognition.face_encodings(nishit_image)[0]

nikhil_image = face_recognition.load_image_file("Nikhil.JPG")
nikhil_encoding = face_recognition.face_encodings(nikhil_image)[0]

known_face_encoding = [
dhruv_encoding,
shan_encoding,
harsh_encoding,
ujjwal_encoding,
nishit_encoding,
nikhil_encoding,
]

known_faces_names = [
"dhruv",
"shan",
"harsh",
"ujjwal",
"nishit",
"nikhil"
]

In [ ]:
students = known_faces_names.copy()

face_locations = []
face_encodings = []
face_names = []
s=True


now = datetime.now()
current_date = now.strftime("%Y-%m-%d")



f = open(current_date+'.csv','w+',newline = '')
lnwriter = csv.writer(f)

while True:
    path = take_photo()
    # path = "/content/Dhruv2.jpeg"
    frame = cv2.imread(path)
    small_frame = cv2.resize(frame,(0,0),fx=0.25,fy=0.25)
    rgb_small_frame = np.ascontiguousarray(small_frame[:, :, ::-1])
    if s:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame,face_locations)
        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encoding,face_encoding)
            name=""
            face_distance = face_recognition.face_distance(known_face_encoding,face_encoding)
            best_match_index = np.argmin(face_distance)
            if matches[best_match_index]:
                name = known_faces_names[best_match_index]

            face_names.append(name)
            if name in known_faces_names:
                font = cv2.FONT_HERSHEY_SIMPLEX
                bottomLeftCornerOfText = (10,100)
                fontScale              = 1.5
                fontColor              = (255,0,0)
                thickness              = 3
                lineType               = 2

                cv2.putText(frame,name+' Present',
                    bottomLeftCornerOfText,
                    font,
                    fontScale,
                    fontColor,
                    thickness,
                    lineType)

                if name in students:
                    students.remove(name)
                    print(name)
                    current_time = now.strftime("%H-%M-%S")
                    lnwriter.writerow([name,current_time])
    #cv2.imshow("attendence system",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
f.close()